<a href="https://colab.research.google.com/github/brotheramin/MachineLearning/blob/main/amir_hossein.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install ta

  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29411 sha256=d918bb2a0478250fac1bc17a4aaf4986a9431d494a79b2df9bdb2cb91c8277f5
  Stored in directory: /root/.cache/pip/wheels/5f/67/4f/8a9f252836e053e532c6587a3230bc72a4deb16b03a829610b
Successfully built ta


In [ ]:
%pip install finpy_tse

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 kB 10.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.8/247.8 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 38.1 MB/s eta 0:00:00
  Created wheel for unsync: filename=unsync-1.4.0-py3-none-any.whl size=4954 sha256=ea43e084c7ae8c9d212bf76e776113f446f486ae2179b9295144d3022be635d1
  Stored in directory: /root/.cache/pip/wheels/bc/6a/4d/c05faa852c92cdf7515eea91a2f07b68645f200e6f85f46961
Successfully built unsync


In [ ]:
import numpy as np
import pandas as pd
import ta
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')
import finpy_tse as fpy
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.feature_selection import SelectKBest, f_regression
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error

In [ ]:
stock = fpy.Get_Price_History(stock='ثتران',
    start_date='1392-01-01',
    end_date='1402-12-28',
    ignore_date=False,
    adjust_price=True,
    show_weekday=False,
    double_date=True)

ConnectTimeout: HTTPConnectionPool(host='old.tsetmc.com', port=80): Max retries exceeded with url: /tsev2/data/search.aspx?skey=%D8%AB%D8%AA%D8%B1%D8%A7%D9%86 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x7ede824ff850>, 'Connection to old.tsetmc.com timed out. (connect timeout=None)'))

In [ ]:
stock = stock.loc[:,["Date", "Adj Close", "Volume","Value","Adj Open","Adj Low","Adj High","No"]]

In [ ]:
stock.rename(columns={"Adj Close": "Close"}, inplace=True)
stock.rename(columns={"Adj Open": "Open"}, inplace=True)
stock.rename(columns={"Adj High": "High"}, inplace=True)
stock.rename(columns={"Adj Low": "Low"}, inplace=True)

In [ ]:
for i in range(1,6):
    stock['Close_lag' + str(i)] = stock['Close'].shift(i)

In [ ]:
def SMA(data, n):
    SMA = data.rolling(window = n).mean()
    return pd.DataFrame(SMA)

n = [26, 50,100]
for i in n:
    stock[f'SMA {i}'] = SMA(stock['Close'], i)

In [ ]:
for i in (12,26,52):
    stock['SMA' + "" + str(i)] = stock.Close.ewm(span=i, min_periods= i).mean()

In [ ]:
def rsi_fun(i):
    rsi = stock.copy()

    rsi["UP"]= np.where( rsi.Close.diff() > 0 , rsi.Close.diff() , 0)
    rsi["DOWN"] = np.where( rsi.Close.diff() < 0 , -rsi.Close.diff() , 0)
    rsi["MA_UP"]= rsi.UP.rolling(i).mean()
    rsi["MA_DOWN"]= rsi.DOWN.rolling(i).mean()
    rsi["RSI" + " " + str(i)]= rsi["MA_UP" ] / (rsi["MA_UP"] + rsi["MA_DOWN"]) *100

    return rsi

In [ ]:
stock["RSI 5"] = rsi_fun(5)["RSI 5"]
stock["RSI 10"] = rsi_fun(10)["RSI 10"]
stock["RSI 15"] = rsi_fun(15)["RSI 15"]
stock["RSI 20"] = rsi_fun(20)["RSI 20"]

In [ ]:
EMA12 = stock['Close'].ewm(span=12, adjust=False).mean()
EMA26 = stock['Close'].ewm(span=26, adjust=False).mean()
stock['MACD'] = EMA12 - EMA26

In [ ]:
high =stock['High']
low =stock['Low']
close =stock['Close']


stock["CCI"] = ta.trend.cci(high, low, close, window=14)

In [ ]:
import pandas as pd
import ta

stock['Stoch'] = ta.momentum.stoch(high=stock['High'], low=stock['Low'], close=stock['Close'])

In [ ]:
def BBANDS(stock, n):
    MA = stock.Close.rolling(n).mean()
    SD = stock.Close.rolling(n).std()
    stock['MiddleBand'] = MA
    stock['UpperBand'] = MA + (2 * SD)
    stock['LowerBand'] = MA - (2 * SD)
    return stock


n = 14
BBANDS = BBANDS(stock, n)

In [ ]:
def ATR(data, period=14):
    high = data['High']
    low = data['Low']
    close = data['Close']

    # Compute True Range
    tr1 = high - low
    tr2 = abs(high - close.shift())
    tr3 = abs(low - close.shift())
    tr = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)

    # Compute ATR
    atr = tr.rolling(window=period).mean()

    return atr

In [ ]:
def calculate_obv(data):
    obv = []
    obv.append(0)

    for i in range(1, len(stock)):
        if data['Close'][i] > data['Close'][i - 1]:
            obv.append(obv[-1] + data['Volume'][i])
        elif data['Close'][i] < data['Close'][i - 1]:
            obv.append(obv[-1] - data['Volume'][i])
        else:
            obv.append(obv[-1])

    return pd.Series(obv, index=stock.index)


In [ ]:
stock['OBV'] = calculate_obv(stock)

In [ ]:
def MFI(data, period=14):
    typical_price = (data['High'] + data['Low'] + data['Close']) / 3
    raw_money_flow = typical_price * data['Volume']
    positive_flow = (typical_price.diff(1) > 0).astype(int) * raw_money_flow
    negative_flow = (typical_price.diff(1) < 0).astype(int) * raw_money_flow

    positive_mf = positive_flow.rolling(window=period).sum()
    negative_mf = negative_flow.rolling(window=period).sum()

    money_flow_ratio = positive_mf / negative_mf
    mfi = 100 - (100 / (1 + money_flow_ratio))

    return mfi


In [ ]:
stock['MFI'] = MFI(stock,14)

In [ ]:
stock['ATR'] = ATR(stock)

In [ ]:
def Momentum(data, period=14):
    close = data['Close']
    momentum = close.diff(period)
    return momentum

In [ ]:
stock['Momentum'] = Momentum(stock,14)

In [ ]:
def ROC(data, period=14):
    close = data['Close']
    roc = (close.diff(period) / close.shift(period)) * 100
    return roc

In [ ]:
stock['ROC'] = ROC(stock,14)

In [ ]:
stock['ADX'] = ta.trend.adx(stock['High'], stock['Low'], stock['Close'], window=14)

In [ ]:
def AD(data):
    high = data['High']
    low = data['Low']
    close = data['Close']
    volume = data['Volume']

    ad_values = ((close - low) - (high - close)) / (high - low) * volume
    ad_values = ad_values.cumsum()

    return ad_values

In [ ]:
stock['AD'] = AD(stock)

In [ ]:
stock['P-SAR'] = ta.trend.psar_up(stock['High'], stock['Low'], stock['Close'], step=0.02, max_step=0.2)

In [ ]:
plt.figure(figsize=(30,15))
corr = stock.drop(columns=["Date"]).corr()
sns.heatmap(data=corr, annot=True, cmap="YlGnBu")

In [ ]:
stock.isna().sum()

In [ ]:
stock.dropna(inplace=True)
len(stock)

In [ ]:
X = stock.drop(['Close', 'Date'], axis=1)
Y = stock['Close']

In [ ]:
X = X.dropna()
Y = Y.loc[X.index]

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns, index=X.index)

In [ ]:
scaled_data = pd.concat([X_scaled_df, Y], axis=1)

In [ ]:
stock['Date'] = pd.to_datetime(stock['Date'], format='%Y-%m-%d')

In [ ]:
scaled_data = pd.concat([scaled_data, stock.loc[X.index, ['Date']]], axis=1)

In [ ]:
split_date = '2022-03-21'
train_data = scaled_data[scaled_data['Date'] < split_date]
test_data = scaled_data[scaled_data['Date'] >= split_date]

In [ ]:
X_train = train_data.drop(['Date', 'Close'], axis=1)
Y_train = train_data['Close']
X_test = test_data.drop(['Date', 'Close'], axis=1)
Y_test = test_data['Close']

In [ ]:
selector = SelectKBest(score_func=f_regression, k=18)
X_train_selected = selector.fit_transform(X_train, Y_train)
X_test_selected = selector.transform(X_test)

In [ ]:
param_grid = {
    'xgbregressor__n_estimators': [800],
    'xgbregressor__max_depth': [20, 50, 80],
    'xgbregressor__learning_rate': [1],
    'xgbregressor__subsample': [0.8],
    'xgbregressor__colsample_bytree': [0.6],
    'xgbregressor__reg_alpha': [0.1,10],
    'xgbregressor__reg_lambda': [2,5],


}


pipeline = make_pipeline(RobustScaler(), XGBRegressor())

grid_search = GridSearchCV(pipeline, param_grid, cv= 15, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search.fit(X_train_selected, Y_train)

In [ ]:
best_params = grid_search.best_params_
best_score = -grid_search.best_score_

In [ ]:
best_model = grid_search.best_estimator_
best_model.fit(X_train_selected, Y_train)

Y_train_predictions = best_model.predict(X_train_selected)
predictions = best_model.predict(X_test_selected)

In [ ]:
print('Best Parameters:', best_params)
print('Best Cross-Validation MSE:', best_score)
print('Best Cross-Validation RMSE:', np.sqrt(best_score))

print('Train MSE:', mean_squared_error(Y_train, Y_train_predictions))
print('Train RMSE:', np.sqrt(mean_squared_error(Y_train, Y_train_predictions)))
print('Train MAE:', metrics.mean_absolute_error(Y_train, Y_train_predictions))
print('Train MAPE:', metrics.mean_absolute_percentage_error(Y_train, Y_train_predictions))

print('Test MSE:', metrics.mean_squared_error(Y_test, predictions))
print('Test RMSE:', np.sqrt(metrics.mean_squared_error(Y_test, predictions)))
print('Test MAE:', metrics.mean_absolute_error(Y_test, predictions))
print('Test MAPE:', metrics.mean_absolute_percentage_error(Y_test, predictions))

In [ ]:
plt.figure(figsize=(40, 7))
plt.plot(Y_test.values, color='blue', label='Actual Stock Price')
plt.plot(predictions, color='red', label='Predicted Stock Price')
plt.title('Stock Price Prediction')
plt.xlabel('Date')
plt.ylabel('Stock Price')
plt.legend()
plt.show()

plt.figure(figsize=(5, 7))
plt.plot(Y_train.values, color='blue', label='Actual Stock Price')
plt.plot(Y_train_predictions, color='red', label='Predicted Stock Price')
plt.title('Stock Price Prediction')
plt.xlabel('Date')
plt.ylabel('Stock Price')
plt.legend()
plt.show()

In [ ]:
features = X_train.columns
importances = rf.feature_importances_
indices = np.argsort(importances)
plt.figure(figsize=(16,10))
plt.title('Feature Importances')
plt.barh(range(len(indices)), importances[indices], color='b', align='center')
plt.yticks(range(len(indices)), [features[i] for i in indices])
plt.xlabel('Relative Importance')
plt.show()